### Closures
A Closure is a function object that remembers values in enclosing scopes even if they are not present in memory.

Closures come in this form:
```python
def closure(internal_state):  # line 1
    def return_function(arguments):  # line 2
        return internal_state combined with arguments  # line 3
    return return_function  # line 4
```

How is this different from a nested function?

In [1]:
def my_function():
    def my_local_function():
        print('I am a local function, local to my_function')
    print('I am not a local function')

A nested function, can take a local variable and use it. But it does not preserve the variable.

In [2]:
def transmit_to_space(message):
    "This is the enclosing function"
    def data_transmitter():
        "The nested function"
        print(message)

    data_transmitter()

print(transmit_to_space("Test message"))

Test message
None


A closure on the otherhand, keeps the local variables preserved.

In [3]:
def transmit_to_space(message):
    "This is the enclosing function"
    def data_transmitter():
        "The nested function"
        print(message)
    return data_transmitter

fun2 = transmit_to_space("Burn the Sun!")
fun2()

Burn the Sun!


ADVANTAGE : Closures can avoid use of global variables and provides some form of data hiding.(Eg. When there are few methods in a class, use closures instead).

Also, Decorators in Python make extensive use of closures.

Closure is commonly used in what is referred to as Function Factory - these are functions that return other functions. The returned functions are specialized. The Function Factory takes in argument(s), creates local function that creates its own argument(s) and also uses the argument(s) passed to the function factory. This is possible with closures

In [4]:
def multiply_by(num):
    def multiply_by_num(k):
        return num * k
    return multiply_by_num
  
five = multiply_by(5)
print(five(2))
print(five(4))
 
decimal = multiply_by(10)
print(decimal(20))
print(decimal(3))

10
20
200
30


Comparing a closure to a class.

Let's say we want to have an average function that works like this:
``` Python
>>> avg(10)
10.0
>>> avg(11)
10.5
>>> avg(12)
11.0
```
We could make a class to do this.

In [6]:
class Average():
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [7]:
avg = Average()
avg(10)

10.0

In [8]:
avg(11)

10.5

In [9]:
avg(12)

11.0

Or we could make a closure.

In [10]:
def make_average():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

In [11]:
avg = make_average()
avg(10)

10.0

In [12]:
avg(11)

10.5

In [13]:
avg(12)

11.0

In [14]:
dir(avg)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

It's obvious where the class Average holds the append values, but it's not obvious where make_average holds these values. 

The closure here allows the function to retain the bindings of the **free variables** that exist when the function is defined. In our case, the *series* is a **free variable** inside *averager* and a **local variable** inside *make_average*. 

In [ ]:
def make_average():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

Let's take a peak under the hood:

In [15]:
avg.__code__.co_varnames

('new_value', 'total')

In [16]:
avg.__code__.co_freevars

('series',)

One more interesting case:

In [17]:
def make_average():
    count, total = 0, 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [18]:
avg = make_average()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

What is happening?

The problem is that `count += 1` means `count = count + 1`, when count is an immutable type. Here we are reassigning count and making it a local variable. Whereas with `series.append()` we we're just altering are mutable type.  

We can fix this with:

In [47]:
def make_average():
    count, total = 0, 0
    
    def averager(new_value):
        nonlocal count, total #nonlocal declaration
        count += 1
        total += new_value
        return total / count
    return averager

In [48]:
dir(make_average)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [24]:
count, total = 0, 0
def make_average():
    
    def averager(new_value):
        global count, total #nonlocal declaration
        count += 1
        total += new_value
        return total / count
    return averager

In [25]:
avg = make_average()
avg(10)

10.0

In [26]:
avg(11)

10.5

### Currying

Currying is a special type of closure.

Currying is a technique where you reduce the number of parameters that function takes, creating a specialized function with one or more of the original parameters set to a particular value. 

An example of currying:

In [30]:
import operator

def curry(func, var):
    y = var
    def f(x):
        return func(x, y)
    return f

double = curry(operator.mul, 2)
add7 = curry(operator.add, 7)
a = double(6)
b = add7(2)
print("Double 6: %i" %a)
print("Add 7 to 2: %i" %b)

Double 6: 12
Add 7 to 2: 9


Currying using the lambda operator

In [31]:
def example(a, b, c):
    return (a, b, c)
  
curried1 = lambda b, c: example(1, b, c)
curried2 = lambda *args: example(1, *args)

In [32]:
curried1(2, 3)

(1, 2, 3)

In [33]:
curried2(2,2)

(1, 2, 2)

In [34]:
def multiply(x):
    def times(y):
        return x * y
    return times
 
times_two = multiply(2)
times_two(10)

20

In [35]:
times_four = multiply(4)
times_four(10)

40

In [36]:
def f(a):
    def g(b):
        def h(c):
            def i(d):
                def j(e):
                    print(a, b, c, d, e)
                return j
            return i
        return h
    return g

f(1)(2)(3)(4)(5)

1 2 3 4 5


### Currying VS partial application

In [37]:
from functools import partial
 
def f(a, b, c, d):
    print(a, b, c, d)
 
g = partial(f, 1, 2, 3)
g(4)

1 2 3 4


In [38]:
g(5)

1 2 3 5


In [39]:
def multiply(x, y):
    """returns x times y"""
    return x * y

In [40]:
double = partial(multiply, y=2)
triple = partial(multiply, y=3)

In [41]:
double(4)

8

In [42]:
triple(4)

12

Could we do this without partial application?

In [43]:
def multiply(x):
    """returns x times y"""
    def times(y):
        return x * y
    return times

In [52]:
dir(multiply)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [60]:
m = multiply(5)
m.__closure__.index

<function tuple.index>

In [44]:
double = multiply(2)
triple = multiply(3)

In [45]:
double(4)

8

In [46]:
triple(4)

12

Partial application will create a manuel curry function for us. 